# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [52]:
# Libraries
import numpy as np 
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [53]:
# your code here
from sqlalchemy import create_engine
driver   = 'mysql+pymysql:'
user     = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136'
database = 'admission_predict'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)
df = pd.read_sql('SELECT * FROM admission_predict', engine)

Remove trailing spaces at the end of the column names if there are any.


In [54]:
# your code here
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

In [55]:
df.columns.str.strip()

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [56]:
# your code here
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3,3.5,8,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.8
3,4,314,103,2,2,3,8.21,0,0.65
4,5,330,115,5,4.5,3,9.34,1,0.9


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [57]:
# your code here
df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [58]:
# your code here
df.set_index('Serial No.', inplace=True)
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3,3.5,8,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.8
4,314,103,2,2,3,8.21,0,0.65
5,330,115,5,4.5,3,9.34,1,0.9
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3,3.5,9.11,1,0.84
383,330,116,4,5,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [59]:
# your code here
df.set_index(['GRE Score', 'CGPA'], inplace=True)
df

,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92
316,8,104,3,3,3.5,1,0.72
322,8.67,110,3,3.5,2.5,1,0.8
314,8.21,103,2,2,3,0,0.65
330,9.34,115,5,4.5,3,1,0.9
...,...,...,...,...,...,...,...
324,9.04,110,3,3.5,3.5,1,0.82
325,9.11,107,3,3,3.5,1,0.84
330,9.45,116,4,5,4.5,1,0.91


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [60]:
df.reset_index(inplace=True)

In [61]:
# your code here
df

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8,104,3,3,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.8
3,314,8.21,103,2,2,3,0,0.65
4,330,9.34,115,5,4.5,3,1,0.9
...,...,...,...,...,...,...,...,...
380,324,9.04,110,3,3.5,3.5,1,0.82
381,325,9.11,107,3,3,3.5,1,0.84
382,330,9.45,116,4,5,4.5,1,0.91
383,312,8.78,103,3,3.5,4,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [62]:
# your code here
# turn CGPA column into floats
df.dtypes

GRE Score             int64
CGPA                 object
TOEFL Score           int64
University Rating     int64
SOP                  object
LOR                  object
Research              int64
Chance of Admit      object
dtype: object

In [63]:
df1 = df.astype({'CGPA': 'float64'})

In [64]:
df1[(df1['CGPA'] > 9) & (df1['Research'] == 1)].count()

GRE Score            101
CGPA                 101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
Research             101
Chance of Admit      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [65]:
# your code here
# turn SOP into a float

df2 = df1.astype({'SOP': 'float64'})

df2[(df2['CGPA'] > 9) & (df2['SOP'] < 3.5)]

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,118,4,3.0,4.5,1,0.91
62,327,9.02,114,3,3.0,3,0,0.61
140,326,9.11,114,3,3.0,3,1,0.83
217,324,9.01,111,4,3.0,3,1,0.82
381,325,9.11,107,3,3.0,3.5,1,0.84


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [66]:
np.std(df2['CGPA'])

0.6016024959756305

In [67]:
(df2['CGPA'] - np.mean(df2['CGPA'])) / np.std(df2['CGPA'])

0      1.750174
1     -0.992501
2      0.121191
3     -0.643433
4      1.234884
         ...   
380    0.736216
381    0.852571
382    1.417729
383    0.304036
384    1.766796
Name: CGPA, Length: 385, dtype: float64

In [68]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return (df3[col] - np.mean(df3[col])) / np.std(df3[col])
    
    
    # your code here

In [69]:
df2.dtypes

GRE Score              int64
CGPA                 float64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                   object
Research               int64
Chance of Admit       object
dtype: object

In [70]:
df3 = df2.astype({'GRE Score':'float64', 'LOR':'float64'})

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [71]:
# your code here
CGPA_std = standardize('CGPA')
GRE_std = standardize('GRE Score')
LOR_std = standardize('LOR')

In [72]:
df3['CGPA_std'] = CGPA_std
df3['GRE_std'] = GRE_std
df3['LOR_std'] = LOR_std
df3

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337.0,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316.0,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322.0,8.67,110,3,3.5,2.5,1,0.8,0.121191,0.456297,-1.039517
3,314.0,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330.0,9.34,115,5,4.5,3.0,1,0.9,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...
380,324.0,9.04,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840
381,325.0,9.11,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840
382,330.0,9.45,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197
383,312.0,8.78,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019


In [73]:
df3.shape[0]

385

We will generate the decision choice at random using the code below. Please run the cell.

In [74]:
# Libraries
from random import choices

In [75]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df3.shape[0])

In [76]:
df3.index

RangeIndex(start=0, stop=385, step=1)

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [77]:
df4 = df3.assign(deciding_column = df3.lookup(df3.index, decision_choice))

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [78]:
# your code here
df4['decisions'] = [1 if x > 0.8 else 0 for x in df4['deciding_column']]
df4

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decisions
0,337.0,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1
1,316.0,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322.0,8.67,110,3,3.5,2.5,1,0.8,0.121191,0.456297,-1.039517,-1.039517,0
3,314.0,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330.0,9.34,115,5,4.5,3.0,1,0.9,1.234884,1.149292,-0.481338,-0.481338,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324.0,9.04,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.076840,0
381,325.0,9.11,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.076840,0
382,330.0,9.45,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729,1
383,312.0,8.78,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.635019,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [79]:
# your code here
df4[df4['decisions'] == 1].count()

GRE Score            86
CGPA                 86
TOEFL Score          86
University Rating    86
SOP                  86
LOR                  86
Research             86
Chance of Admit      86
CGPA_std             86
GRE_std              86
LOR_std              86
deciding_column      86
decisions            86
dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [80]:
# your code here
df4.columns = df4.columns.str.lower().str.replace(' ', '_')
df4.columns

Index(['gre_score', 'cgpa', 'toefl_score', 'university_rating', 'sop', 'lor',
       'research', 'chance_of_admit', 'cgpa_std', 'gre_std', 'lor_std',
       'deciding_column', 'decisions'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [83]:
condlist = [df4["university_rating"] >= 4]
choicelist = [df4["gre_score"] + 10]
df4["adjusted_gre"] = np.select(condlist, choicelist, default = df4["gre_score"])

In [84]:
df4

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decisions,adjusted_gre
0,337.0,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.193197,1,347.0
1,316.0,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0,316.0
2,322.0,8.67,110,3,3.5,2.5,1,0.8,0.121191,0.456297,-1.039517,-1.039517,0,322.0
3,314.0,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0,314.0
4,330.0,9.34,115,5,4.5,3.0,1,0.9,1.234884,1.149292,-0.481338,-0.481338,0,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324.0,9.04,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,0.076840,0,324.0
381,325.0,9.11,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,0.076840,0,325.0
382,330.0,9.45,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,1.417729,1,340.0
383,312.0,8.78,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,0.635019,0,312.0


In [89]:
labels1 = ['Low', 'Medium', 'High', 'Very High']
bins = pd.cut(df4['adjusted_gre'], 4, labels=labels1,retbins=True)
bins

(0      Very High
 1         Medium
 2           High
 3         Medium
 4      Very High
          ...    
 380         High
 381         High
 382    Very High
 383       Medium
 384    Very High
 Name: adjusted_gre, Length: 385, dtype: category
 Categories (4, object): [Low < Medium < High < Very High],
 array([289.94, 305.  , 320.  , 335.  , 350.  ]))